# STAT 207 Homework 6 [25 points]

## Deeper Dive into Hypothesis Testing and Introduction to Linear Regression

Due: Thursday, March 13, end of day (11:59 pm CT)

Late submissions accepted until Friday, March 14 at noon

<hr>

## Imports 

Run the following code cell to import the necessary packages into the file.  You may import additional packages, as needed for this assignment.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

## Case Study: County Housing Information

The provided **county.csv** file contains various statistics about the population of every county in the United States.  You can read more about this dataset and its variables here:

https://www.openintro.org/data/index.php?data=county

**Observational Unit**: Each row contains data recorded for a county

**Variables**: 

- In question 1, we will focus on the `multi_unit` variable in the data, which reports the percent of housing units in each county that are in multi-unit structures (e.g. apartments).  
- In question 2, we will add the `homeownership` variable, which records the rate of home ownership.  

Below, we read in the data as `df` for later analysis.

In [ ]:
df = pd.read_csv('county.csv', na_values = ['data unavailable'])

## 1. Deeper Dive Into p-Values and Significance Levels [5 points] 

**a)** Calculate the mean multi-unit housing rate for the data.

**b)** We will start by creating a sampling distribution of the mean multi-unit housing rate for 5 randomly selected counties.  Record 5000 repetitions in your simulated sampling distribution.

**c)** Now, we will use the following hypotheses as the framework for our next parts:

$H_0: \mu = 12.32$

$H_a: \mu > 12.32$

First, take a random sample of size 5 from the population of counties **without replacement**.  Within the `sample` function, use the argument `random_state = 207`.  Using this random sample along with the sampling distribution from part **b**, calculate the p-value for the test above.

**d)** Now, we'd like to explore the behavior of a p-value.  In order to do this, we will calculate p-values for many different random samples of size 5.  In essence, we will repeat the process from **part c** many times, although we will **not** use the `random_state` argument for our repeated samples.

Specifically, you should:
- gather a random sample of size 5 without replacement (which mimics a realistic sampling scheme in this scenario),
- use the sampling distribution from part **b** to calculate the corresponding p-value, and
- save the p-value for future analysis

Repeat this process 500 times, so that you have 500 different p-values in your data to explore.

*Hint:* As a first step, determine the appropriate code to calculate the p-value for a single random sample of size 5.  Then, once you have code that works for a single random sample, generalize it so that you can repeat this for 500 different samples.

**e)** Finally, suppose that we consider Champaign and its four neighboring counties (Vermilion, Ford, Piatt, and Douglas) as a sample of counties from the US.  First, calculate the sample mean multi-unit rate of these five counties from Illinois.  Then, using your simulated sampling distribution, calculate the p-value based on these five counties.

**Tip**: You may want to review Homework 4.  We can use the **`&`** ("and") operator to indicate that we want **both** conditions on either side of the operator to be met.  We can use the **`|`** ("or") operator to indicate that we want **at least one** of the conditions to be met.  We can also chain these operators together if we need to represent more complex operations.

## 2. Linear Regression [4 points]

How does the rate of homeownership affect the percent of housing units that are in multi-unit structures?

**a)** Generate a visualization to observe the relationship between the rate of homeownership and the percentage of multi-unit structure housing in the county.  Be sure to place each variable on the correct axes based on the underlying question above.

**b)** Fit a linear model that predicts the `multi_unit` from `homeownership`.  Print the summary of the model.

Remember to keep all your cells and hit the save icon above periodically to checkpoint (save) your results on your local computer. Once you are satisified with your results restart the kernel and run all (Kernel -> Restart & Run All). **Make sure nothing has changed**. Checkpoint and exit (File -> Save and Checkpoint + File -> Close and Halt). Follow the instructions on the Homework 6 Canvas Assignment to submit your notebook to GitHub.